# [과제 2] 회귀분석
### - Ch 1, Ch 2를 토대로 자유롭게 회귀분석과 회귀진단을 진행해주세요.
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

In [138]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

# Data 

데이터 출처 : https://www.kaggle.com/avikasliwal/used-cars-price-prediction 

< y > 
* **Price** : The price of the used car in INR Lakhs.



< X > 
* Name : The brand and model of the car
* Location : The location in which the car is being sold or is available for purchase.
* Year : The year or edition of the model.
* Kilometers_Driven : The total kilometres driven in the car by the previous owner(s) in KM.
* Fuel_Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)
* Transmission : The type of transmission used by the car. (Automatic / Manual)
* Owner_Type : Whether the ownership is Firsthand, Second hand or other.
* Mileage : The standard mileage offered by the car company in kmpl or km/kg
* Engine : The displacement volume of the engine in CC.
* Power : The maximum power of the engine in bhp.
* Seats : The number of seats in the car.
* New_Price : The price of a new car of the same model.

In [157]:
# Load Data 
data = pd.read_csv("assignment2_data.csv")
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [140]:
data.isnull().sum() 

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [158]:
#new_price 컬럼은 결측치가 너무 많아서 삭제, 나머지 삭제된 변수들은 의미없어보이는 변수들
data=data.drop(['New_Price','Unnamed: 0','Name','Location'],axis=1)
data

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...
6014,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,4.75
6015,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,4.00
6016,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,2.90
6017,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,2.65


In [159]:
string_vals = ['Power', 'Engine', 'Mileage'] #유의미해보이는 문자형 변수들

for col in string_vals:
    data[col] = pd.to_numeric(data[col].str.split().str.get(0),errors='coerce')  # 변수명 뒤에 문자들 떼어내기

In [160]:
data=data.dropna(axis=0)
data

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75
1,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
2,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50
3,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
4,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...
6014,2014,27365,Diesel,Manual,First,28.40,1248.0,74.00,5.0,4.75
6015,2015,100000,Diesel,Manual,First,24.40,1120.0,71.00,5.0,4.00
6016,2012,55000,Diesel,Manual,Second,14.00,2498.0,112.00,8.0,2.90
6017,2013,46000,Petrol,Manual,First,18.90,998.0,67.10,5.0,2.65


In [161]:
data=data.reset_index(drop=True)
data

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75
1,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50
2,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50
3,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00
4,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74
...,...,...,...,...,...,...,...,...,...,...
5867,2014,27365,Diesel,Manual,First,28.40,1248.0,74.00,5.0,4.75
5868,2015,100000,Diesel,Manual,First,24.40,1120.0,71.00,5.0,4.00
5869,2012,55000,Diesel,Manual,Second,14.00,2498.0,112.00,8.0,2.90
5870,2013,46000,Petrol,Manual,First,18.90,998.0,67.10,5.0,2.65


In [163]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(data[['Mileage','Engine','Power',	'Seats']])
a=mMscaler.transform(data[['Mileage','Engine','Power',	'Seats']])

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(a)

mMscaled_data.columns=['Mileage','Engine','Power',	'Seats']
b=data[['Year','Kilometers_Driven','Fuel_Type','Transmission','Owner_Type','Price']]
data=pd.concat([b, mMscaled_data],axis=1)
data

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Price,Mileage,Engine,Power,Seats
0,2010,72000,CNG,Manual,First,1.75,0.793083,0.069594,0.045569,0.375
1,2015,41000,Diesel,Manual,First,12.50,0.586464,0.178266,0.174971,0.375
2,2011,46000,Petrol,Manual,First,4.50,0.542636,0.106997,0.103652,0.375
3,2012,87000,Diesel,Manual,First,6.00,0.619261,0.116115,0.103766,0.625
4,2013,40670,Diesel,Automatic,Second,17.74,0.453190,0.250093,0.202739,0.375
...,...,...,...,...,...,...,...,...,...,...
5867,2014,27365,Diesel,Manual,First,4.75,0.846750,0.116115,0.075694,0.375
5868,2015,100000,Diesel,Manual,First,4.00,0.727490,0.092296,0.069989,0.375
5869,2012,55000,Diesel,Manual,Second,2.90,0.417412,0.348716,0.147965,0.750
5870,2013,46000,Petrol,Manual,First,2.65,0.563506,0.069594,0.062571,0.375


In [164]:
data.isnull().sum() 

Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Price                0
Mileage              0
Engine               0
Power                0
Seats                0
dtype: int64

In [153]:
data['Fuel_Type'].unique()


array(['CNG', 'Diesel', 'Petrol', 'LPG'], dtype=object)

In [47]:
data['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [25]:
data['Owner_Type'].unique()

array(['First', 'Second', 'Fourth & Above', 'Third'], dtype=object)

In [165]:
cats = ['Fuel_Type', 'Transmission', 'Owner_Type'] #범주형 변수만 리스트로 저장하기

In [166]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import itertools

def pre_reg_transform(input_df,train_df,categoricals,dependent_var=None): #범주형 변수를 원핫 인코더로 변경하기
    df = input_df
    # Convert Categorical Values
    for colname in categoricals:
        train_df[colname] = train_df[colname].astype('category')
        df[colname] = df[colname].astype('category')
    
    c = df.select_dtypes(include=['category']) #카테고리형 데이터만 호출하기
    _c = train_df.select_dtypes(include=['category'])

    one_hot_enc = OneHotEncoder(sparse = False,handle_unknown='ignore') #원핫 인코딩
    one_hot_enc.fit(_c) #원핫 인코딩
    
    c_trns = one_hot_enc.transform(c) #원핫인코딩 시켜주기
    c_df = pd.DataFrame(c_trns, columns=one_hot_enc.get_feature_names()) #원핫인코딩 시킨값 데이터프레임으로 만들기

    _df = df.drop(columns=['Price', *categoricals]).reset_index() if dependent_var else df.drop(columns=categoricals).reset_index() #카테고리 형 변수들 없애기
    X = pd.concat([_df, c_df],axis=1) #원핫인코딩시킨값이랑 기존 변수들 합치기
    y = df[dependent_var] if dependent_var else None
    
    return X, y, df, one_hot_enc, c_trns

In [167]:
X, y, _, enc, transform = pre_reg_transform(input_df=data,train_df=data,
                         categoricals=cats,
                         dependent_var='Price')

In [168]:
data #기존 데이터

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Price,Mileage,Engine,Power,Seats
0,2010,72000,CNG,Manual,First,1.75,0.793083,0.069594,0.045569,0.375
1,2015,41000,Diesel,Manual,First,12.50,0.586464,0.178266,0.174971,0.375
2,2011,46000,Petrol,Manual,First,4.50,0.542636,0.106997,0.103652,0.375
3,2012,87000,Diesel,Manual,First,6.00,0.619261,0.116115,0.103766,0.625
4,2013,40670,Diesel,Automatic,Second,17.74,0.453190,0.250093,0.202739,0.375
...,...,...,...,...,...,...,...,...,...,...
5867,2014,27365,Diesel,Manual,First,4.75,0.846750,0.116115,0.075694,0.375
5868,2015,100000,Diesel,Manual,First,4.00,0.727490,0.092296,0.069989,0.375
5869,2012,55000,Diesel,Manual,Second,2.90,0.417412,0.348716,0.147965,0.750
5870,2013,46000,Petrol,Manual,First,2.65,0.563506,0.069594,0.062571,0.375


In [169]:
X #범주형데이터에 원핫인코딩을 적용한 것

,index,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,x0_CNG,x0_Diesel,x0_LPG,x0_Petrol,x1_Automatic,x1_Manual,x2_First,x2_Fourth & Above,x2_Second,x2_Third
0,0,2010,72000,0.793083,0.069594,0.045569,0.375,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1,2015,41000,0.586464,0.178266,0.174971,0.375,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,2,2011,46000,0.542636,0.106997,0.103652,0.375,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,2012,87000,0.619261,0.116115,0.103766,0.625,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,4,2013,40670,0.453190,0.250093,0.202739,0.375,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,5867,2014,27365,0.846750,0.116115,0.075694,0.375,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5868,5868,2015,100000,0.727490,0.092296,0.069989,0.375,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5869,5869,2012,55000,0.417412,0.348716,0.147965,0.750,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5870,5870,2013,46000,0.563506,0.069594,0.062571,0.375,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [170]:
X=X.drop(['index','Year'],axis=1) #무의미한 변수 삭제하기

In [171]:
# splitting the data in 70:30 ratio of train to test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

regression_model = LinearRegression(fit_intercept=True)
regression_model.fit(X_train, y_train) #회귀모델에 핏하기

y_pred = regression_model.predict(X_test)


In [174]:
# Errors
print('-----------Errors------------')
print('R-squared training data: ', regression_model.score(X_train, y_train))
print('R-squared test data: ', regression_model.score(X_test, y_test))
print('MSE: ', mean_squared_error(y_test, y_pred))
print('MAE: ', mean_absolute_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

-----------Errors------------
R-squared training data:  0.6312198722551117
R-squared test data:  0.6470607239312729
MSE:  42.67747846466687
MAE:  3.9879832438343406
RMSE:  6.5328002008837585


In [175]:
#결과: MSE가 너무 높게 나왔다... EDA를 시간을 들여 더 꼼꼼하게 해야할 것 같다.